In [1]:
import os
import pandas as pd
import json

def update_config(config_path, output_dir, ground_truth_path, det_path, save_path):
    with open(config_path, 'r') as file:
        config = json.load(file)

    config['output_dir'] = output_dir
    config['ground_truth_path'] = ground_truth_path
    config['det_path'] = det_path

    with open(save_path, 'w') as file:
        json.dump(config, file, indent=4)

def run_eval_on_all_files(folder, eval_path, gt_path, config_path, eval_name):
    # List all files in the folder that match the pattern
    files = [f for f in os.listdir(folder) if f.startswith("res_df_ep") and f.endswith(".tsv")]

    # Sort files to ensure correct processing order (by epoch number)
    files.sort()

    for file in files:
        # Extract the epoch number from the file name
        epoch = int(file.split("res_df_ep")[1].split(".tsv")[0])

        # Paths for current iteration
        det_path = os.path.join(folder, file)
        output_dir = os.path.join(folder, f"{eval_name}_ep{epoch}")

        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Read ground truth and detection data
        gt_df = pd.read_csv(gt_path, sep="\t")
        det_df = pd.read_csv(det_path, sep="\t")

        # Filter ground truth data
        cut_gt_df = gt_df[gt_df["name"].isin(det_df["name"])]
        cut_gt_path = os.path.join(output_dir, "cut_gt.tsv")
        cut_gt_df.to_csv(cut_gt_path, sep="\t", index=False)

        # Update the configuration file
        config_new_path = os.path.join(output_dir, "wrapper.json")
        update_config(config_path, output_dir, cut_gt_path, det_path, config_new_path)

        # Run the evaluation command
        command = (
            f"export PYTHONPATH=$PYTHONPATH:{eval_path} && "
            f"cd {eval_path} && "
            f"/home/b2b/anaconda3/envs/od_arthur/bin/python3 worker.py --config {config_new_path}"
        )
        os.system(command)

# Example usage:
run_eval_on_all_files(
     folder="/home/b2b/arthur/git/codetr/work_dirs/default_600_crop_01_lr_01_wd_20_peds_data",
     eval_path="/home/b2b/arthur/git/EvalBin",
     gt_path="/home/b2b/arthur/data/921_hard_images/gt.tsv",
     config_path="/home/b2b/arthur/git/EvalBin/wrappers/wrapper_config_with_ignore.json",
     eval_name="NewEval_wi"
 )


/home/b2b/arthur/git/EvalBin/src/preprocess/preprocess.py:167: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[f"{group}_ignore_label"].replace({True: 1, False: 0}, inplace=True)
/home/b2b/arthur/git/EvalBin/src/preprocess/preprocess.py:167: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[f"{group